In [ ]:
https://blog.csdn.net/sofuzi/article/details/80825197
1、排序。耗时太长，况且只需找前k个，不用全部排序

2、局部淘汰法。用一个容器保存前10000个数，然后将剩余的所有数字一一与容器内的最小数字相比，如果所有后续的元素都比容器内的10000个数还小，
那么容器内这个10000个数就是最大10000个数。如果某一后续元素比容器内最小数字大，则删掉容器内最小元素，并将该元素插入容器，最后遍历完这
1亿个数，得到的结果容器中保存的数即为最终结果了。此时的时间复杂度为O（n+m^2），其中m为容器的大小。

这个容器可以用（小顶堆）最小堆来实现。
这个算法优点是性能尚可，空间复杂度低，IO读取比较频繁，对系统压力大。

3、第三种方法是分治法，即大数据里最常用的MapReduce。

a、将100亿个数据分为1000个大分区，每个区1000万个数据
b、每个大分区再细分成100个小分区。总共就有1000*100=10万个分区
c、计算每个小分区上最大的1000个数。

为什么要找出每个分区上最大的1000个数？举个例子说明，全校高一有100个班，我想找出全校前10名的同学，很傻的办法就是，把高一100个班的同学
成绩都取出来，作比较，这个比较数据量太大了。应该很容易想到，班里的第11名，不可能是全校的前10名。也就是说，不是班里的前10名，就不可能
是全校的前10名。因此，只需要把每个班里的前10取出来，作比较就行了，这样比较的数据量就大大地减少了。我们要找的是100亿中的最大1000个数，
所以每个分区中的第1001个数一定不可能是所有数据中的前1000个。

d、合并每个大分区细分出来的小分区。每个大分区有100个小分区，我们已经找出了每个小分区的前1000个数。将这100个分区的1000*100个数合并，
找出每个大分区的前1000个数。

e、合并大分区。我们有1000个大分区，上一步已找出每个大分区的前1000个数。我们将这1000*1000个数合并，找出前1000.这1000个数就是所有数据
中最大的1000个数。

（a、b、c为map阶段，d、e为reduce阶段）

4、Hash法。如果这1亿个书里面有很多重复的数，先通过Hash法，把这1亿个数字去重复，这样如果重复率很高的话，会减少很大的内存用量，从而缩小
运算空间，然后通过分治法或最小堆法查找最大的10000个数。

对于海量数据处理，思路基本上是：必须分块处理，然后再合并起来。（可以先去重复，进行数据清洗）

In [ ]:
# 待解决的自动换行问题，以下两个链接都尝试了，还是不行
# https://www.zhihu.com/question/56110064
# https://blog.csdn.net/tripple_x/article/details/48978211

In [1]:
# -*- coding: utf-8 -*-
# 俄罗斯方块，download from the net  ，没有安装pygame
import pygame, random

__author__ = {
    'name': "东方鹗",
    'github': "https://github.com/eastossifrage",
    'B 站主页': "https://space.bilibili.com/194359739",
    '知乎专栏': "https://zhuanlan.zhihu.com/ousi-python",
    '简书专栏': "https://www.jianshu.com/c/2036276cf0e8",
    'version': "1.0"
}
# 定义几个颜色     R   G   B
bgcolor       = ( 30,  25,  25) # 背景设置为黑色
yellow        = (255, 245,  40) # 黄色
light_yellow  = (175, 175,  20)
white         = (255, 245, 245)
gray          = (185, 185, 185)
black         = (  0,   0,   0)
red           = (155,   0,   0)
light_red     = (175,  20,  20)
green         = (  0, 155,   0)
light_green   = ( 20, 175,  20)
blue          = (  0,   0, 155)
light_blue    = ( 20,  20, 175)

# 定义颜色元组

colors = (yellow, gray, black, red, green, blue)

# 定义屏幕大小
s_width = 800
s_height = 600

# 定义俄罗斯方块游戏区域的大小
a_width = 364
a_height = 544

# 定义正方形的大小
edge = 20

# 定义游戏区域左上角坐标
coordinate = ((s_width-a_width)//4, s_height-a_height)

# 定义游戏区域中的小方格的行列数
box_row = a_height//edge # 小方格的每列个数，也就是行数
box_col = a_width//edge  # 小方格的每行个数，也就是列数

#定义游戏区域中的小方格的开始位置
inital_pos = (0, int(box_col//2))

# 以下是七种形状的拼版按照小方格的位置所对应的坐标
I = [[(0, -1), (0, 0), (0, 1), (0, 2)],
     [(-1, 0), (0, 0), (1, 0), (2, 0)]]
J = [[(-2, 0), (-1, 0), (0, 0), (0, -1)],
     [(-1, 0), (0, 0), (0, 1), (0, 2)],
     [(0, 1), (0, 0), (1, 0), (2, 0)],
     [(0, -2), (0, -1), (0, 0), (1, 0)]]
L = [[(-2, 0), (-1, 0), (0, 0), (0, 1)],
     [(1, 0), (0, 0), (0, 1), (0, 2)],
     [(0, -1), (0, 0), (1, 0), (2, 0)],
     [(0, -2), (0, -1), (0, 0), (-1, 0)]]
O = [[(0, 0), (0, 1), (1, 0), (1, 1)]]
S = [[(-1, 0), (0, 0), (0, 1), (1, 1)],
     [(1, -1), (1, 0), (0, 0), (0, 1)]]
T = [[(0, -1), (0, 0), (0, 1), (-1, 0)],
     [(-1, 0), (0, 0), (1, 0), (0, 1)],
     [(0, -1), (0, 0), (0, 1), (1, 0)],
     [(-1, 0), (0, 0), (1, 0), (0, -1)]]
Z = [[(0, -1), (0, 0), (1, 0), (1, 1)],
     [(-1, 0), (0, 0), (0, -1), (1, -1)]]

# 定义拼版元组
# shapes_tuple = (I, J, L, O, S, T, Z)
shapes_tuple = (J, L, S, T, Z)

# 定义刷新频率
fps = 15 #设置屏幕刷新率
def box(rect, color):
    ''' 功能：画出用于组成图形的单个正方形
        参数：rect: 该参数为一个pygame.Rect对象在游戏区域的相对坐标
              color：小方格的颜色
    '''
    pygame.Surface.fill(screen, color, rect) # 填充方格内的颜色
    pygame.draw.rect(screen, white, rect, 1)  # 画出方格的线条

def area():
    ''' 功能：画出游戏区域
    '''
    rect = pygame.Rect(coordinate, (a_width, a_height))
    pygame.draw.rect(screen, light_blue, rect, 5)          # 画出游戏区域

def rect_matrix():
    ''' 功能：以游戏区域为基础，画出用于呈现方块图形的矩阵。
    '''
    # 下面一行列表推导的代码用于构造小方格的二维列表
    return [[pygame.Rect((coordinate[0] + x*edge + 2),
              (coordinate[1] + y*edge + 2), edge, edge)
            for x in range(box_col)] for y in range(box_row)]

def color_matrix():
    ''' 功能：以游戏区域为基础，画出用于呈现方块颜色的矩阵。'''
    # 下面一行列表推导的代码用于构造小方格颜色的二维列表，默认是 None
    return [[None for x in range(box_col)] for y in range(box_row)]


def show_shape(rect_matrix, row, col, shape, color):
    '''功能：显示游戏区域内的四格拼版。
        参数：matrix: 游戏区域的网格的坐标
             row：拼版的中心点相对背景网格的横坐标
             col：拼版的中心点相对背景网格的纵坐标
             shape: 四格拼版的小方格相对于中心点的坐标
             color：拼版的颜色
    '''
    for x in shape:
        if row+x[0] >= 0: # 舍弃中心点以上部分，防止从最底部出现
            box(rect_matrix[row+x[0]][col+x[1]], color)

def conflict(color_matrix, row, col, shape):
    '''功能：检测游戏区域内的四格拼版是否与底部边界或其它已存在的拼版有冲突。
        参数：matrix: 游戏区域的网格的坐标
             row：拼版的中心点相对背景网格的横坐标
             col：拼版的中心点相对背景网格的纵坐标
             shape: 四格拼版的小方格相对于中心点的坐标
    '''
    for x in shape:
        if row+x[0] >= box_row:                   # 检测四格拼版是否与
            return True                           # 游戏区域内的底部边界有冲突
        if row+x[0] >= 0 and \
        color_matrix[row+x[0]][col+x[1]] != None: # 检测四格拼板中的小方格是否
            return True                           # 与当前位置有冲突

    return False

def clear_line(color_matrix):
    '''功能：清空已连成一线的行'''
    num = 0
    for x in color_matrix:
        if None not in x:
            num = color_matrix.index(x)
    for i in reversed(range(1, num+1)): # 需要倒序输出
        color_matrix[i] = color_matrix[i-1]


if __name__ == "__main__":
    pygame.init()  # 使用 pygame 之前先进行初始化
    clock=pygame.time.Clock()  # 定义时钟
    screen = pygame.display.set_mode((s_width, s_height), pygame.RESIZABLE, 32)
    # 上面一行代码的功能为设置屏幕大小，此实例为 800*600，
    # 第二个参数是 RESIZABLE，表示打开一个可以改变大小的窗口，
    # 第三个参数是 32 位真彩色。

    pygame.display.set_caption("俄罗斯方块") # 设置窗体名称

    r_mtx = rect_matrix()
    c_mtx = color_matrix()
    row, col = inital_pos
    color = random.choice(colors)
    shapes = random.choice(shapes_tuple)
    shape =  random.choice(shapes)
    shape_num = shapes.index(shape)  # 获取四格拼板的形状列表的下标

    while True:
        #pygame.time.delay(25)
        screen.fill(bgcolor)  # 表示填充背景颜色。
        for event in pygame.event.get():
            if event.type == pygame.QUIT: # 检测退出动作
                exit()

        area()

        for r in range(box_row):
            for c in range(box_col):
                if c_mtx[r][c] is not None:
                    box(r_mtx[r][c], c_mtx[r][c])

        shape_left = min(shape, key=lambda x:x[1])[1]   # 拼版的最左边坐标值
        shape_right = max(shape, key=lambda x:x[1])[1]  # 拼版的最右边坐标值

        cft = conflict(c_mtx, row, col, shape)

        keys = pygame.key.get_pressed()  # 检测键盘动作

        if keys[pygame.K_LEFT] and not cft:      # 使用向左键来控制方块向左移动
            if col + shape_left > 0:
                col -= 1
            if conflict(c_mtx, row, col, shape): # 如果冲突
                col += 1                         # 回退一格

        if keys[pygame.K_RIGHT] and not cft:     # 使用向左键来控制方块向右移动
            if col + shape_right < box_col-1:
                col += 1
            if conflict(c_mtx, row, col, shape): # 如果冲突
                col -= 1                         # 回退一格

        if keys[pygame.K_a] and not cft:         # 使用 W 键将拼版左转
            shape_num -= 1
            shape = shapes[shape_num%len(shapes)] # 选择新拼版
            shape_left = min(shape, key=lambda x:x[1])[1]   # 新拼版的最左边坐标值
            shape_right = max(shape, key=lambda x:x[1])[1]  # 新拼版的最右边坐标值
            if col+shape_left < 0 or \
            col+shape_right > box_col-1 or \
            conflict(c_mtx, row, col, shape): # 如果冲突、超出左或右边界
            # 要注意 or 逻辑的短路顺序
                shape_num += 1                            # 回退到上一个拼版
                shape = shapes[shape_num%len(shapes)]    # 选择原拼版

        if keys[pygame.K_d] and not cft:         # 使用 D 键将拼版右转
            shape_num += 1
            shape = shapes[shape_num%len(shapes)]
            shape_left = min(shape, key=lambda x:x[1])[1]   # 拼版的最左边坐标值
            shape_right = max(shape, key=lambda x:x[1])[1]  # 拼版的最右边坐标值
            if col+shape_left < 0 or \
            col+shape_right > box_col-1 or \
            conflict(c_mtx, row, col, shape): # 如果冲突、超出左或右边界
            # 要注意 or 逻辑的短路顺序
                shape_num -= 1                            # 回退到上一个拼版
                shape = shapes[shape_num%len(shapes)]     # 选择原拼版


        if cft:
            for x in shape:
                if row+x[0] > 0:  # 舍弃中心点以上部分，防止从最底部出现
                    c_mtx[row+x[0]-1][col+x[1]] = color
            if row != 0:    # 检测拼版重叠的位置是否在第一行，如果不是，生成新拼版
                row, col = inital_pos                # 设置新拼版的起始位置
                color = random.choice(colors)        # 重新选择颜色
                shapes = random.choice(shapes_tuple)
                shape =  random.choice(shapes)       # 重新选择拼版
            else:          # 检测拼版重叠的位置是否在第一行，如果是，游戏结束
                print("GAME OVER")
                break
        else:
            clear_line(c_mtx)                         # 清空已填满的行
            show_shape(r_mtx, row, col, shape, color) # 显示游戏区域内的四格拼版
            row += 1
        clock.tick(fps)
        pygame.display.update()

ModuleNotFoundError: No module named 'pygame'